In [12]:
import pandas as pd

In [ ]:
def prepare_data(filename, term):
    due_dates = []
    courses = []
    titles = []

    df = pd.read_json(filename)

    for title in df['title']:
        # Has a due date
        if str(title).startswith(('Sun', 'Mon', 'Tue', "Wed", "Thu", "Fri", "Sat")):
            subs = str(title).split(' ')
            due_dates.append(subs[1])
            courses.append(subs[2])
            titles.append(" ".join(subs[3::]))
        # Does not have a due date
        else:
            subs = str(title).split(' ')
            due_dates.append(None)
            courses.append(subs[0])
            titles.append(" ".join(subs[1::]))

    df['due_date'] = due_dates
    df['course'] = courses
    df['title'] = titles

    df = df[['start_date', 'end_date', 'course', 'duration', 'title', 'due_date']]
    df['start_date'] = pd.to_datetime(df['start_date'])
    year = df.iloc[0]['start_date'].year
    df['end_date'] = pd.to_datetime(df['end_date'])
    df['due_date'] = pd.to_datetime(df['due_date'] + f"/{year}")
    df['duration'] = pd.to_numeric(df['duration'].apply(lambda x: x.split(" ")[0]))
    df['term'] = term
    df["day of week"] = df["start_date"].apply(lambda x: x.day_name())
    df["day of week"] = pd.Categorical(df["day of week"], categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], ordered=True)
    if term != 'fa23':
        df["week"] = df["start_date"].apply(lambda x: x.week - df.iloc[0]['start_date'].week + 1)
    else:
        df["week"] = df["start_date"].apply(lambda x: x.week - 34 + 1)
    df = df[["term", "week", "day of week", "course", "duration", "start_date", "end_date", 'due_date', 'title']]
    return df


In [14]:
fa23 = prepare_data("../data/raw/structured_export/fa23.json", 'fa23')
fa23['course'] = fa23['course'].replace({"Hist": "HIST 164", "Math": "MATH 241", "Phys": "PHYS 213/214", "Stat": "STAT 107"})
fa23 = fa23[fa23['course'] != 'AIS']

sp24 = prepare_data("../data/raw/structured_export/sp24.json", 'sp24')
sp24['course'] = sp24['course'].replace({"Math": "MATH 257", "Anth": "ANTH 103", "Stat": "STAT 207", "Geol": "GEOL 107", "Astro": "ASTR 210"})

fa24 = prepare_data("../data/raw/structured_export/fa24.json", 'fa24')
fa24['course'] = fa24['course'].replace({"Chem": "CHEM 102/103", "Aces": "ACES 179", "IS": "IS 477", "Geol": "GEOL 208", "Astro": "ASTR 310"})
fa24 = fa24[fa24['course'] != 'Research']

sp25 = prepare_data("../data/raw/structured_export/sp25.json", 'sp25')
sp25['course'] = sp25['course'].replace({"Chem": "CHEM 104/105", "CS": "CS 307", "Astro": "ASTR 405", "Geol": "GEOL 432"})


/var/folders/41/_gkgvhb94wd4156zplzr4cg00000gn/T/ipykernel_65961/1777197631.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['start_date'] = pd.to_datetime(df['start_date'])
/var/folders/41/_gkgvhb94wd4156zplzr4cg00000gn/T/ipykernel_65961/1777197631.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['end_date'] = pd.to_datetime(df['end_date'])
/var/folders/41/_gkgvhb94wd4156zplzr4cg00000gn/T/ipykernel_65961/1777197631.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['start_date'] = pd.to_datetime(df['start_date'])
/var/folders/41/_gkgvhb94wd4156zplzr4cg00

In [15]:
master = pd.concat([fa23, sp24, fa24, sp25])
master.to_csv("../data/master/studytime2.csv", index=False)

In [16]:
sp25

,term,week,day of week,course,duration,start_date,end_date,due_date,title
0,sp25,1,Tuesday,Research,60,2025-01-21 10:30:00,2025-01-21 11:30:00,NaT,
1,sp25,1,Wednesday,ASTR 405,15,2025-01-22 16:30:00,2025-01-22 16:45:00,2025-01-24,PS1
2,sp25,1,Wednesday,Research,120,2025-01-22 18:30:00,2025-01-22 20:30:00,NaT,
3,sp25,1,Thursday,CHEM 104/105,90,2025-01-23 09:15:00,2025-01-23 10:45:00,2025-02-05,Lab Policy & Safety Quiz
4,sp25,1,Thursday,ASTR 405,45,2025-01-23 10:45:00,2025-01-23 11:30:00,2025-01-23,Read Ch1
5,sp25,1,Thursday,Research,75,2025-01-23 14:15:00,2025-01-23 15:30:00,NaT,
6,sp25,1,Thursday,Research,135,2025-01-23 18:30:00,2025-01-23 20:45:00,NaT,
7,sp25,1,Friday,CHEM 104/105,15,2025-01-24 12:15:00,2025-01-24 12:30:00,2025-01-23,PL
8,sp25,1,Saturday,Research,240,2025-01-25 12:00:00,2025-01-25 16:00:00,NaT,
9,sp25,1,Sunday,CHEM 104/105,15,2025-01-26 13:30:00,2025-01-26 13:45:00,2025-02-05,Lab Practice Report
